In [ ]:
pip install pyserial

In [ ]:
pip install pyserial PyQt5 opencv-python numpy

In [ ]:

import sys
import cv2
import numpy as np
import serial
import time
from PyQt5 import QtWidgets, QtGui, QtCore

class VideoThread(QtCore.QThread):
    original_frame_signal = QtCore.pyqtSignal(np.ndarray)
    tracked_frame_signal = QtCore.pyqtSignal(np.ndarray)
    center_info_signal = QtCore.pyqtSignal(str)
    esp32_error_signal = QtCore.pyqtSignal(str)
    esp32_output_signal = QtCore.pyqtSignal(str)

    def __init__(self, cam_index=0, serial_port='COM3', baudrate=115200):
        super().__init__()
        self._run_tracking = False
        self.cam_index = cam_index
        self.brightness = 0
        self.saturation = 0
        self.hue_min = 40
        self.hue_max = 80
        self.s_min = 50
        self.v_min = 50
        self.capture = None

        try:
            self.ser = serial.Serial(serial_port, baudrate, timeout=1)
            time.sleep(2)
        except:
            self.ser = None

    def run(self):
        self.capture = cv2.VideoCapture(self.cam_index)
        while True:
            ret, frame = self.capture.read()
            if not ret:
                continue

            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            hsv[..., 2] = np.clip(hsv[..., 2] + self.brightness, 0, 255)
            hsv[..., 1] = np.clip(hsv[..., 1] + self.saturation, 0, 255)
            adjusted_frame = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
            self.original_frame_signal.emit(adjusted_frame.copy())

            tracked_frame = adjusted_frame.copy()
            info_text = "中心位置: -, 偏差值: -"

            if self._run_tracking:
                h_min = min(self.hue_min, self.hue_max)
                h_max = max(self.hue_min, self.hue_max)
                lower = np.array([h_min, self.s_min, self.v_min])
                upper = np.array([h_max, 255, 255])
                mask = cv2.inRange(hsv, lower, upper)

                contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if contours:
                    c = max(contours, key=cv2.contourArea)
                    x, y, w, h = cv2.boundingRect(c)
                    cx = x + w // 2
                    cy = y + h // 2
                    img_center_x = frame.shape[1] // 2
                    error = img_center_x - cx
                    cv2.rectangle(tracked_frame, (x, y), (x + w, y + h), (0, 255, 0), -1)
                    info_text = f"中心位置: ({cx}, {cy}), 偏差值: {error}"

                    if self.ser and self.ser.is_open:
                        try:
                            self.ser.write(f"{error}\n".encode())
                        except:
                            pass

            self.center_info_signal.emit(info_text)
            self.tracked_frame_signal.emit(tracked_frame)

            if self.ser and self.ser.in_waiting:
                try:
                    feedback = self.ser.readline().decode().strip()
                    if "," in feedback and "偏差" in feedback:
                        parts = feedback.replace("偏差:", "").replace("輸出:", "").split(",")
                        if len(parts) == 2:
                            self.esp32_error_signal.emit(parts[0])
                            self.esp32_output_signal.emit(parts[1])
                    else:
                        pass
                except:
                    pass

    def start_tracking(self):
        self._run_tracking = True

    def stop_tracking(self):
        self._run_tracking = False

class ColorTrackingUI(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("追蹤系統")
        self.setGeometry(100, 100, 800, 1150)
        self.thread = None
        self.initUI()

    def initUI(self):
        layout = QtWidgets.QVBoxLayout()

        self.original_label = QtWidgets.QLabel()
        self.original_label.setFixedSize(640, 360)
        self.tracked_label = QtWidgets.QLabel()
        self.tracked_label.setFixedSize(640, 360)
        self.center_info = QtWidgets.QLabel("中心位置: -, 偏差值: -")
        self.esp32_error = QtWidgets.QLabel("ESP32 偏差值: -")
        self.esp32_output = QtWidgets.QLabel("ESP32 輸出值: -")

        layout.addWidget(QtWidgets.QLabel("原始畫面", alignment=QtCore.Qt.AlignCenter))
        layout.addWidget(self.original_label)
        layout.addWidget(QtWidgets.QLabel("追蹤畫面", alignment=QtCore.Qt.AlignCenter))
        layout.addWidget(self.tracked_label)
        layout.addWidget(self.center_info)
        layout.addWidget(self.esp32_error)
        layout.addWidget(self.esp32_output)

        self.cam_combo = QtWidgets.QComboBox()
        self.cam_combo.addItems(["0", "1", "2"])

        self.serial_input = QtWidgets.QLineEdit("COM8")

        self.hue_label = QtWidgets.QLabel()
        self.hue_label.setPixmap(QtGui.QPixmap(self.generate_hue_pixmap(640, 20)))

        self.h_min_slider = QtWidgets.QSlider(QtCore.Qt.Horizontal)
        self.h_min_slider.setRange(0, 179)
        self.h_min_slider.setValue(40)

        self.h_max_slider = QtWidgets.QSlider(QtCore.Qt.Horizontal)
        self.h_max_slider.setRange(0, 179)
        self.h_max_slider.setValue(80)

        self.s_min_slider = QtWidgets.QSlider(QtCore.Qt.Horizontal)
        self.s_min_slider.setRange(0, 255)
        self.s_min_slider.setValue(50)

        self.v_min_slider = QtWidgets.QSlider(QtCore.Qt.Horizontal)
        self.v_min_slider.setRange(0, 255)
        self.v_min_slider.setValue(50)

        self.brightness_slider = QtWidgets.QSlider(QtCore.Qt.Horizontal)
        self.brightness_slider.setRange(-100, 100)
        self.brightness_slider.setValue(0)

        self.saturation_slider = QtWidgets.QSlider(QtCore.Qt.Horizontal)
        self.saturation_slider.setRange(-100, 100)
        self.saturation_slider.setValue(0)

        self.reset_button = QtWidgets.QPushButton("重置亮度/飽和度")

        self.start_button = QtWidgets.QPushButton("啟動追蹤")
        self.stop_button = QtWidgets.QPushButton("停止追蹤")

        self.color_combo = QtWidgets.QComboBox()
        self.color_combo.addItems(["請選擇預設色域", "紅色", "綠色", "藍色", "黃色"])
        self.color_combo.currentTextChanged.connect(self.apply_preset_color)

        form = QtWidgets.QFormLayout()
        form.addRow("選擇攝影機：", self.cam_combo)
        form.addRow("ESP32 串口：", self.serial_input)
        form.addRow("套用常用顏色：", self.color_combo)
        form.addRow("Hue 色彩條：", self.hue_label)
        form.addRow("H最小值：", self.h_min_slider)
        form.addRow("H最大值：", self.h_max_slider)
        form.addRow("S最小值：", self.s_min_slider)
        form.addRow("V最小值：", self.v_min_slider)
        form.addRow("亮度調整：", self.brightness_slider)
        form.addRow("飽和度調整：", self.saturation_slider)
        form.addRow(self.reset_button)
        form.addRow(self.start_button, self.stop_button)

        layout.addLayout(form)
        self.setLayout(layout)

        self.cam_combo.currentIndexChanged.connect(self.init_video_thread)
        self.reset_button.clicked.connect(self.reset_brightness_saturation)
        self.start_button.clicked.connect(self.start_tracking)
        self.stop_button.clicked.connect(self.stop_tracking)

    def reset_brightness_saturation(self):
        self.brightness_slider.setValue(0)
        self.saturation_slider.setValue(0)

    def generate_hue_pixmap(self, width, height):
        hue_strip = np.zeros((height, width, 3), dtype=np.uint8)
        for x in range(width):
            hue = int((x / width) * 179)
            hue_strip[:, x] = [hue, 255, 255]
        bgr = cv2.cvtColor(hue_strip, cv2.COLOR_HSV2BGR)
        qimg = QtGui.QImage(bgr.data, width, height, 3 * width, QtGui.QImage.Format_RGB888).rgbSwapped()
        return QtGui.QPixmap.fromImage(qimg)

    def init_video_thread(self):
        if self.thread:
            self.thread.terminate()

        cam_index = int(self.cam_combo.currentText())
        serial_port = self.serial_input.text()
        self.thread = VideoThread(cam_index, serial_port)

        self.brightness_slider.valueChanged.connect(lambda val: setattr(self.thread, 'brightness', val))
        self.saturation_slider.valueChanged.connect(lambda val: setattr(self.thread, 'saturation', val))
        self.h_min_slider.valueChanged.connect(lambda val: setattr(self.thread, 'hue_min', val))
        self.h_max_slider.valueChanged.connect(lambda val: setattr(self.thread, 'hue_max', val))
        self.s_min_slider.valueChanged.connect(lambda val: setattr(self.thread, 's_min', val))
        self.v_min_slider.valueChanged.connect(lambda val: setattr(self.thread, 'v_min', val))

        self.thread.original_frame_signal.connect(self.update_original)
        self.thread.tracked_frame_signal.connect(self.update_tracked)
        self.thread.center_info_signal.connect(self.update_info)
        self.thread.esp32_error_signal.connect(self.update_error)
        self.thread.esp32_output_signal.connect(self.update_output)

        self.thread.start()

    def start_tracking(self):
        if self.thread:
            self.thread.start_tracking()

    def stop_tracking(self):
        if self.thread:
            self.thread.stop_tracking()

    def update_original(self, cv_img):
        qt_img = self.convert_cv_qt(cv_img)
        self.original_label.setPixmap(qt_img)

    def update_tracked(self, cv_img):
        qt_img = self.convert_cv_qt(cv_img)
        self.tracked_label.setPixmap(qt_img)

    def update_info(self, text):
        self.center_info.setText(text)

    def update_error(self, text):
        self.esp32_error.setText(f"ESP32 偏差值: {text}")

    def update_output(self, text):
        self.esp32_output.setText(f"ESP32 輸出值: {text}")

    def apply_preset_color(self, color):
        presets = {
            "紅色": (0, 10),
            "黃色": (20, 35),
            "綠色": (40, 85),
            "藍色": (100, 130)
        }
        if color in presets:
            hmin, hmax = presets[color]
            self.h_min_slider.setValue(hmin)
            self.h_max_slider.setValue(hmax)

    def convert_cv_qt(self, cv_img):
        rgb_image = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb_image.shape
        bytes_per_line = ch * w
        qt_image = QtGui.QImage(rgb_image.data, w, h, bytes_per_line, QtGui.QImage.Format_RGB888)
        return QtGui.QPixmap.fromImage(qt_image).scaled(640, 360, QtCore.Qt.KeepAspectRatio)

def run_app():
    app = QtWidgets.QApplication(sys.argv)
    window = ColorTrackingUI()
    window.show()
    sys.exit(app.exec_())

run_app()
